<h1> Investigation of Data Aggregation in Pandas

In [11]:
# Load the data into the pandas dataframe
import pandas as pd


df = pd.read_csv("data_sp18.csv") # Modify to correct data location
# print(df.head())
print(df.columns)
# help(df)

Index(['Term Code', 'College Code', 'Department Code', 'CRN', 'Campus Code',
       'Subject Code', 'Course Number', 'Section Number', 'Section Title',
       'Instructor 1 ID', 'Instructor 1 First Name', 'Instructor 1 Last Name',
       'Question Number', 'Question', 'Mean', 'Median', 'Standard Deviation',
       'Department Mean', 'Department Median', 'Similar College Mean',
       'Similar College Median', 'College Mean', 'College Median',
       'Percent Rank - Department', 'Percent Rank - College', 'Responses',
       'Percent #1', 'Percent #2', 'Percent #3', 'Percent #4', 'Percent #5',
       'ZScore - College', 'ZScore - College Similar Sections', 'Course Level',
       'Section Size'],
      dtype='object')


In [298]:
import numpy as np
import pandas as pd

df = pd.read_csv("data/data_sp18.csv") # Modify to correct data location

# Initialize the aggregated dataframe by copying the base data frame
ag_df = df.copy()

# Drop the unnecessary columns
ag_df.drop(['Department Code', 'Question Number','Section Number','Responses', 'CRN','Campus Code','Question', 'Mean', 'Median', 'Standard Deviation', 'Department Mean', 'Department Median', 'Similar College Mean', 'College Mean', 'College Median', 'Percent Rank - Department', 'Percent Rank - College', 'Percent #1', 'Percent #2', 'Percent #3', 'Percent #4', 'Percent #5', 'ZScore - College', 'ZScore - College Similar Sections', 'Course Level', 'Section Size', 'Similar College Median'], axis=1, inplace = True)


# Add in the columns to be filled with the aggregated values
ag_df.insert(3,'Avg Department Rating', 0.0)
ag_df.insert(4,'SD Department Rating', 0.0)
ag_df.insert(7,'Avg Course Rating', 0.0)
ag_df.insert(8,'SD Course Rating', 0.0)
ag_df.insert(12, 'Avg Instructor Rating In Section', 0.0)
ag_df.insert(13, 'SD Instructor Rating In Section', 0.0)
ag_df.insert(14, 'Num Responses', 0)
# # Swap the columns Section Number and Section Title
# swap_col_order = list(ag_df.columns)
# # swap_col_order[6] = 'Instructor ID'
# # swap_col_order[9] = 'Section Number'
# ag_df = ag_df.reindex(columns = swap_col_order)

# Rename the Instructor 1 ID, Instructor 1 First Name, Last Name columns
ag_df.rename(columns = {'Section Title':'Course Title', 'Responses':'Num Responses','Instructor 1 ID': 'Instructor ID', 'Instructor 1 First Name':'Instructor First Name', 'Instructor 1 Last Name':'Instructor Last Name'}, inplace= True)

# Remove the repeat rows that will occur because we are taking 1-10 question responses down to 1
ag_df.drop_duplicates(subset = ag_df.columns.drop('Course Title'), inplace = True)

# Now we have the column headings we want
# Fill the columns with correct values
# print(ag_df.head())
# ag_df[(ag_df['Subject Code']=='DSA') & (ag_df['Course Number']==4413)]
import sys
# Read in the question mappings values from the mappings.yaml
import yaml
with open('mappings.yaml') as f:
    # use safe_load instead load
    mappings = yaml.safe_load(f)
    question_weighting = mappings['Instructor_question_weighting']

# Lets fill the average instructor ranking in each section, i.e. the combined rating for each question per section
for subject in df['Subject Code'].unique(): # Iterate over all subjects (test case - for subject in ['DSA']:)
    for course in df[(df['Subject Code']==subject)]['Course Number'].unique(): # Iterate over courses with the desired subject 
        for instructor in df[(df['Subject Code']==subject) & (df['Course Number']==course)]['Instructor 1 ID'].unique(): # Iterate over instructors with desired subject and course number
            # Modify the subset based on the desired instructor (see section index above)
            subset = df[(df['Subject Code']==subject) & (df['Course Number']==course) & (df['Instructor 1 ID']==instructor)]        
            if len(subset)!=0: 
                # Compute the combined mean and standard deviation of the questions
                # Input the standard deviation, mean, number of responses, and the question number mapped to the weights for each subject-course-instructor combination
#                 print(len(subset['Standard Deviation']))
                instructor_mean, instructor_sd = combine_standard_deviations(subset['Standard Deviation'], subset['Mean'], subset['Responses'], subset['Question Number'].map(arg=question_weighting))
                
                # Set the combined mean and combined sd value into the aggregated dataframe
                # Find the row of interest in the aggregated df
                ag_df_section_row = ag_df[(ag_df['Subject Code']==subject) & (ag_df['Course Number']==course) & (ag_df['Instructor ID']==instructor)].index.tolist()
                if len(ag_df_section_row)!=1:
                    print('Aggregated Dataframe contains incorrect number of entries (number entries =' + str(len(ag_df_section_row))+ ') for subject: '+ str(subject)+ ', course: '+ str(course)+ ', and instructor: '+ str(instructor))
                else:
                    # Fill the Instructor Ratings Columns
                    ag_df.at[ag_df_section_row[0], 'Avg Instructor Rating In Section'] = instructor_mean
                    ag_df.at[ag_df_section_row[0], 'SD Instructor Rating In Section'] = instructor_sd
                    
                    # Fill the Num Responses column, based on the minimum number of responses of the group of questions
                    # if they teach two or more courses, add the number of students in each course to get the total number
                    total_responses = 0
                    if int(len(subset['Responses'])/5) <1:
                        print('Less than 5 questions are available for subject: '+ str(subject)+ ', course: '+ str(course)+ ', and instructor: '+ str(instructor))
                    for i in range(int(len(subset['Responses'])/5)):
                        total_responses = total_responses + list(subset['Responses'])[5*i]
                    # Set the Num Responses in the agg df equal to the total responses in this course
                    ag_df.at[ag_df_section_row[0], 'Num Responses'] = total_responses
                    
        # Back to Course level of tree, now that we've filled out the instructor level info
        # Modify the dataframe subset that consists only of the entries with the desired course (see course index above)
        # Note that now our subset consists of aggregated data from all instructors within the desired course
        subset = ag_df[(ag_df['Subject Code']==subject) & (ag_df['Course Number']==course)]
        # Compute the combined mean and standard deviation of all of the instructors within the course
        #### IMPORTANT #### NO POPULATION-BASED OR OTHER WEIGHTING USED IN THE CALCULATION OF SD AND AVERAGE COURSE RATING
        
        course_mean, course_sd = combine_standard_deviations(subset['SD Instructor Rating In Section'], subset['Avg Instructor Rating In Section'], np.ones(len(subset['SD Instructor Rating In Section'])), np.ones(len(subset['SD Instructor Rating In Section'])))
        # Find the row of interest in the desired df
        ag_df_course_rows = ag_df[(ag_df['Subject Code']==subject) & (ag_df['Course Number']==course)].index.tolist()
        # Fill the Course ratings columns
        ag_df.at[ag_df_course_rows, 'Avg Course Rating'] = course_mean
        ag_df.at[ag_df_course_rows, 'SD Course Rating'] = course_sd
        
    # Back to Department level of tree, now that we've filled out the instructor and course level info
    # Modify the dataframe subset that consists only of the entries with the desired subject(see course index above)
    # Note that now our subset consists of aggregated data from all instructors and courses within the desired subject/department
    subset = ag_df[(ag_df['Subject Code']==subject)]
    # Compute the combined mean and standard deviation of all of the courses within the department
    #### IMPORTANT #### NO POPULATION-BASED OR OTHER WEIGHTING USED IN THE CALCULATION OF SD AND AVERAGE COURSE RATING

    department_mean, department_sd = combine_standard_deviations(subset['SD Course Rating'], subset['Avg Course Rating'], np.ones(len(subset['Avg Course Rating'])), np.ones(len(subset['Avg Course Rating'])))
    # Find the row of interest in the desired df
    ag_df_course_rows = ag_df[(ag_df['Subject Code']==subject)].index.tolist()
    # Fill the Course ratings columns
    ag_df.at[ag_df_course_rows, 'Avg Department Rating'] = department_mean
    ag_df.at[ag_df_course_rows, 'SD Department Rating'] = department_sd


In [300]:
ag_df

,Term Code,College Code,Subject Code,Avg Department Rating,SD Department Rating,Course Number,Course Title,Avg Course Rating,SD Course Rating,Instructor ID,Instructor First Name,Instructor Last Name,Avg Instructor Rating In Section,SD Instructor Rating In Section,Num Responses
0,201720,EN,DSA,4.131492,1.101032,4413,Algorithm Analysis,3.513333,0.956524,112111442,S,Lakshmivarahan,3.513333,0.956524,15
10,201720,EN,DSA,4.131492,1.101032,5001,Data Analytics and Media,4.254777,0.740368,112129082,Doyle,Yoon,4.254777,0.740368,8
20,201720,EN,DSA,4.131492,1.101032,5011,Introduction to R,4.290221,1.038015,112979983,Wayne,Stewart,4.290221,1.038015,16
30,201720,EN,DSA,4.131492,1.101032,5113,Analytics & Metaheuristics,4.580488,0.617664,112998038,Charles,Nicholson,4.580488,0.617664,41
45,201720,EN,DSA,4.131492,1.101032,5203,Time Series Analysis,2.943750,1.492490,112111442,S,Lakshmivarahan,2.943750,1.492490,8
50,201720,EN,DSA,4.131492,1.101032,5900,Professional Practice,4.754543,0.450802,112123488,Randa,Shehab,4.754543,0.450802,11
55,201720,EN,DSA,4.131492,1.101032,5970,Bayesian Statistics,4.583335,0.711256,112979983,Wayne,Stewart,4.583335,0.711256,9
65,201720,EN,ENGR,4.080546,1.225309,1411,Freshman Engineering Exper,3.666723,1.262679,112281778,Laura,Wesson,2.736666,1.114719,15
70,201720,EN,ENGR,4.080546,1.225309,1411,Freshman Engineering Exper,3.666723,1.262679,113363969,Kathy,Volz,4.165625,1.022928,16
75,201720,EN,ENGR,4.080546,1.225309,1411,Disc-ENGR 1411,3.666723,1.262679,112123488,Randa,Shehab,4.097879,1.092837,31


In [297]:
# print(ag_df.head(20)['Instructor Last Name'])
df[115:125]

,Term Code,College Code,Department Code,CRN,Campus Code,Subject Code,Course Number,Section Number,Section Title,Instructor 1 ID,...,Responses,Percent #1,Percent #2,Percent #3,Percent #4,Percent #5,ZScore - College,ZScore - College Similar Sections,Course Level,Section Size
115,201720,EN,EN00,37591,N,ENGR,2002,25,Professional Development,112115442,...,5,0.0,0.0,0.0,0.0,100.0,0.73290,0.73290,All,All
116,201720,EN,EN00,37591,N,ENGR,2002,25,Professional Development,112115442,...,5,0.0,0.0,0.0,0.0,100.0,0.81770,0.81770,All,All
117,201720,EN,EN00,37591,N,ENGR,2002,25,Professional Development,112115442,...,5,0.0,0.0,0.0,0.0,100.0,0.73171,0.73171,All,All
118,201720,EN,EN00,37591,N,ENGR,2002,25,Professional Development,112115442,...,5,0.0,0.0,0.0,0.0,100.0,0.84085,0.84085,All,All
119,201720,EN,EN00,37591,N,ENGR,2002,25,Professional Development,112115442,...,5,0.0,0.0,0.0,0.0,100.0,0.76363,0.76363,All,All
120,201720,EN,EN00,37591,N,ENGR,2002,25,Professional Development,112128340,...,5,0.0,0.0,0.0,20.0,80.0,0.53812,0.53812,All,All
121,201720,EN,EN00,37591,N,ENGR,2002,25,Professional Development,112128340,...,5,0.0,0.0,0.0,20.0,80.0,0.64488,0.64488,All,All
122,201720,EN,EN00,37591,N,ENGR,2002,25,Professional Development,112128340,...,5,0.0,0.0,0.0,20.0,80.0,0.52816,0.52816,All,All
123,201720,EN,EN00,37591,N,ENGR,2002,25,Professional Development,112128340,...,5,0.0,0.0,0.0,40.0,60.0,0.49999,0.49999,All,All
124,201720,EN,EN00,37591,N,ENGR,2002,25,Professional Development,112128340,...,5,0.0,0.0,0.0,40.0,60.0,0.40675,0.40675,All,All


In [268]:
ag_df.columns.drop('Course Title')
ag_df.columns


Index(['Term Code', 'College Code', 'Subject Code', 'Avg Department Rating',
       'SD Department Rating', 'Course Number', 'Course Title',
       'Avg Course Rating', 'SD Course Rating', 'Instructor ID',
       'Instructor First Name', 'Instructor Last Name',
       'Avg Instructor Rating In Section', 'SD Instructor Rating In Section',
       'Num Responses'],
      dtype='object')

The formula that we will use for computing and combining the ratings from the available question data is shown, without weighting, below. The image is available from [here](https://www.researchgate.net/post/How_to_combine_standard_deviations_for_three_groups)

![sd_image](https://www.researchgate.net/profile/Twan_Ten_Haaf/post/How_to_combine_standard_deviations_for_three_groups/attachment/59d64a7479197b80779a4bdd/AS%3A475377563901954%401490350251587/download/Grouped_SD.jpg)

In the above formula, the i index corresponds to the question number. In addition to the $ n_i$ and $(n-1)_i$ terms, our formula uses a w_i term for the weighting of each question; That is, each question response and standard deviation is weighted *not only* by the number of responses but also by the weight for that specific question. These formulae will be used throughout the data aggregation for combining data for each section and also for combining standard deviations for each course and each department. Also, the implemented formula does not use the $n-1$, but instead just uses n (minor error for n>5)

In [192]:
def combine_means(mean_list, pop_list, weight_list):
    '''
    This function takes lists of means, population sizes, and weights for each population, and combines the result into a single mean value.
    * All lists must be the same length
    '''
    import numpy as np
    mean_list = np.array(mean_list)
    pop_list = np.array(pop_list)
    weight_list = np.array(weight_list)
    # Check for equal sized lists
    if not len(mean_list)==len(pop_list)==len(weight_list):
        print('All input lists to the function -- combine_standard_deviations -- must be of the same length.')
        # Proceed with the program
    # Combine the population-and-weight-modulated means
    mean = np.sum(mean_list*pop_list*weight_list)/(np.sum(pop_list*weight_list))
    return mean
    

def combine_standard_deviations(sd_list, mean_list,pop_list, weight_list):
    '''
    This function will take lists of standard deviations, means, population sizes, and weights for each list unit. The function
    will combine the lists to produce a standard deviation for the group, based on the input parameters. Formula for combining the SD taken from the below link:
    
    https://www.researchgate.net/post/How_to_combine_standard_deviations_for_three_groups
    
    * All lists must be same length
    '''
    import numpy as np
    # Convert all input lists into numpy arrays
    sd_list = np.array(sd_list)
    mean_list = np.array(mean_list)
    pop_list = np.array(pop_list)
    weight_list = np.array(weight_list)
    # Check for equal sized lists
    if not len(sd_list) == len(mean_list)==len(pop_list)==len(weight_list):
        print('All input lists to the function -- combine_standard_deviations -- must be of the same length.')
    # Proceed with the program
    # Compute the weighted mean of the populations
    pop_mean = combine_means(mean_list, pop_list, weight_list)
    # Compute the deviance
    deviance = np.sum((pop_list)*(weight_list)*(sd_list**2) + (pop_list*weight_list)*(mean_list - pop_mean)**2)
    # Compute the standard deviation
    sd = np.sqrt(deviance/(np.sum(pop_list*weight_list)))
    return pop_mean,sdd



In [143]:
print(combine_means([2,9], [10,10], [0.5,1]))
combine_standard_deviations([4,6],[50,9], [47,100], [1,1])

# Validated the above functions for combined sd and combined means with this website, albeit there was No entry or validation for the weighting
# https://www.statstodo.com/CombineMeansSDs_Pgm.php

6.666666666666667
pop mean is 22.108843537414966


19.880297993804273

In [8]:
import yaml
with open('mappings.yaml') as f:
    # use safe_load instead load
    mappings = yaml.safe_load(f)
Term_code_dict = mappings['Term_code_dict']
College_code_dict = mappings['College_code_dict']
Department_code_dict = mappings['Department_code_dict']
Department_code_dict


{'EN00': 'ENGR', 'EN01': 'AME', 'EN02': 'CH E', 'EN03': 'CEES'}

In [94]:
import numpy as np
a = np.array(np.array(np.array([1,2,3])))

a**2


array([1, 4, 9])

In [1]:
import data_aggregation
import pandas as pd
df = pd.read_csv("data/data_sp18.csv") # Modify to correct data location
data_aggregation.aggregate_data(df)

Holy shit iterate


,Term Code,College Code,Subject Code,Avg Department Rating,SD Department Rating,Course Number,Course Title,Avg Course Rating,SD Course Rating,Instructor ID,Instructor First Name,Instructor Last Name,Avg Instructor Rating In Section,SD Instructor Rating In Section,Num Responses
0,201720,EN,DSA,4.131492,1.101032,4413,Algorithm Analysis,3.513333,0.956524,112111442,S,Lakshmivarahan,3.513333,0.956524,15
10,201720,EN,DSA,4.131492,1.101032,5001,Data Analytics and Media,4.254777,0.740368,112129082,Doyle,Yoon,4.254777,0.740368,8
20,201720,EN,DSA,4.131492,1.101032,5011,Introduction to R,4.290221,1.038015,112979983,Wayne,Stewart,4.290221,1.038015,16
30,201720,EN,DSA,4.131492,1.101032,5113,Analytics & Metaheuristics,4.580488,0.617664,112998038,Charles,Nicholson,4.580488,0.617664,41
45,201720,EN,DSA,4.131492,1.101032,5203,Time Series Analysis,2.943750,1.492490,112111442,S,Lakshmivarahan,2.943750,1.492490,8
50,201720,EN,DSA,4.131492,1.101032,5900,Professional Practice,4.754543,0.450802,112123488,Randa,Shehab,4.754543,0.450802,11
55,201720,EN,DSA,4.131492,1.101032,5970,Bayesian Statistics,4.583335,0.711256,112979983,Wayne,Stewart,4.583335,0.711256,9
65,201720,EN,ENGR,0.000000,0.000000,1411,Freshman Engineering Exper,0.000000,0.000000,112281778,Laura,Wesson,0.000000,0.000000,0
70,201720,EN,ENGR,0.000000,0.000000,1411,Freshman Engineering Exper,0.000000,0.000000,113363969,Kathy,Volz,0.000000,0.000000,0
75,201720,EN,ENGR,0.000000,0.000000,1411,Disc-ENGR 1411,0.000000,0.000000,112123488,Randa,Shehab,0.000000,0.000000,0
